In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

df = pd.read_csv("/content/drive/MyDrive/intern_work/titanic-dataset/tested.csv")

In [3]:
numerical_features = ['Age', 'Fare']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['Sex', 'Embarked']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


X = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Survived'])
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

print(X_train_preprocessed[:5])
print(y_train[:5])

[[ 0.19632439 -0.41182271  0.          1.          0.          0.
   1.        ]
 [-0.41817392 -0.09227106  0.          1.          0.          0.
   1.        ]
 [-0.18773705 -0.45140739  0.          1.          0.          1.
   0.        ]
 [-0.41817392  0.78462533  0.          1.          0.          0.
   1.        ]
 [-0.80223537 -0.45500537  0.          1.          0.          0.
   1.        ]]
336    0
31     0
84     0
287    0
317    0
Name: Survived, dtype: int64


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

rf_model = RandomForestClassifier(random_state=42)

rf_model.fit(X_train_preprocessed, y_train)

y_pred = rf_model.predict(X_test_preprocessed)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        34

    accuracy                           1.00        84
   macro avg       1.00      1.00      1.00        84
weighted avg       1.00      1.00      1.00        84

Confusion Matrix:
 [[50  0]
 [ 0 34]]


In [6]:
def predict_survival(passenger_details):
    passenger_df = pd.DataFrame([passenger_details])

    passenger_preprocessed = preprocessor.transform(passenger_df)

    prediction = rf_model.predict(passenger_preprocessed)

    return prediction[0]


passenger_details = {
    'Pclass': 3,
    'Sex': 'male',
    'Age': 22,
    'SibSp': 1,
    'Parch': 0,
    'Fare': 50.5,
    'Embarked': 'S'
}

prediction = predict_survival(passenger_details)

print("Survived" if prediction == 1 else "Did not survive")

Survived
